In [50]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
#for installing all the libraries
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install transformers
!pip install simpletransformers
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git


In [ ]:
#for checking gpu details if present any on machine
!nvidia-smi

Mon Aug  5 17:34:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0              30W /  70W |  10225MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#for loading evaluate function
!pip install evaluate

In [ ]:
!cd /content/drive/MyDrive/ClickBait

In [ ]:
!pip install tensorflow
!pip install tensorflow-text


In [ ]:
#for importing necessary all the libraries
import json
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from tqdm import tqdm
import torch
import nltk
import evaluate

In [ ]:
# to load .jsonl dataset files in the code and returning dataframe of important columns for generation task.(mainly two columns, content and their respective spoilers)
def load_dataset(file_name):
    data = []
    with open(file_name, encoding='utf8') as f:
        for line in f:
            example = json.loads(line)
            post_text = example['postText'][0]
            title = example['targetTitle']
            paragraphs = ' '.join(example['targetParagraphs'])
            spoiler = example['spoiler'][0]
            data.append({'text': post_text + ' - ' + title + paragraphs, 'spoiler': spoiler})
    return pd.DataFrame(data)
train_data = load_dataset(r'/content/drive/MyDrive/ClickBait/task-2-clickbait-detection-msci-641-s-24/train.jsonl')
validation_data = load_dataset(r'/content/drive/MyDrive/ClickBait/task-2-clickbait-detection-msci-641-s-24/val.jsonl')


In [ ]:
# converting both dataframes to numpy to further apply preprocessing on it.
train_data = np.array(train_data)
validation_data = np.array(validation_data)

In [ ]:
# For removing all the extra special characters from the data
characters = ['!','"','#','$','%','&','(',')','*','+','/',':',';','<','=','>','@','^','`','|','~','\t','[',']','{','}','\\','.','-']
for i in range(len(train_data)):
    for j in characters:
        train_data[i][0] = train_data[i][0].replace(j,"")
        train_data[i][1] = train_data[i][1].replace(j,"")

for i in range(len(validation_data)):
    for j in characters:
        validation_data[i][0] = validation_data[i][0].replace(j,"")
        validation_data[i][1] = validation_data[i][1].replace(j,"")


print(len(train_data))
print(len(validation_data))

train_data = pd.DataFrame(train_data)
validation_data = pd.DataFrame(validation_data)

3200
400


In [ ]:
# Converting dataframes and storing them as .csv file
train_data.to_csv(r'/content/drive/MyDrive/ClickBait/task-2-clickbait-detection-msci-641-s-24/train.csv', index=False)
validation_data.to_csv(r'/content/drive/MyDrive/ClickBait/task-2-clickbait-detection-msci-641-s-24/val.csv', index=False)

In [ ]:
# loading dataset in the form which is acceptable to pretrained model
from datasets import load_dataset

dataset_file = '/content/drive/MyDrive/ClickBait/task-2-clickbait-detection-msci-641-s-24/train.csv'
dataset_file_1 = '/content/drive/MyDrive/ClickBait/task-2-clickbait-detection-msci-641-s-24/val.csv'
train_data = load_dataset('csv', data_files=dataset_file)
val_data = load_dataset('csv', data_files=dataset_file_1)


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(train_data['train'][1]['1'])

2070


In [ ]:
# loading pretrained model bart-base on gpu of the server along with it's tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model_ckpt = "facebook/bart-large"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

cuda


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Creating embeddings of the data
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['0'] , max_length = 1024, truncation = True , padding= "max_length")

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['1'], max_length = 50, truncation = True , padding= "max_length")

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

train = train_data.map(convert_examples_to_features, batched = True)
val = val_data.map(convert_examples_to_features, batched = True)


Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [ ]:
print(val_data)

DatasetDict({
    train: Dataset({
        features: ['0', '1'],
        num_rows: 400
    })
})


In [ ]:
#Intializing data collector
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [ ]:
#Defining training arguments
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

trainer_args = Seq2SeqTrainingArguments(
    output_dir='/content/drive/MyDrive/ClickBait/Model_BART', num_train_epochs=10, warmup_steps=0,
    per_device_train_batch_size=2, per_device_eval_batch_size=2,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='epoch', save_steps=1e6,
    gradient_accumulation_steps=16
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Passing trainer arguments
trainer = Seq2SeqTrainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=train['train'],
                  eval_dataset=val['train'])

In [ ]:
# for starting training of the model
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.384400,0.277076
2,0.217500,0.211343
3,0.139900,0.200704
4,0.081900,0.241397
5,0.068700,0.222361
6,0.051600,0.238697
7,0.044500,0.252408
8,0.024600,0.252487
9,0.029300,0.285158
10,0.020800,0.275233


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=1000, training_loss=0.4362225283682346, metrics={'train_runtime': 15093.9789, 'train_samples_per_second': 2.12, 'train_steps_per_second': 0.066, 'total_flos': 6.9347347267584e+16, 'train_loss': 0.4362225283682346, 'epoch': 10.0})

In [ ]:
# For saving model after training
model_pegasus.save_pretrained(r"/content/drive/MyDrive/ClickBait/Model_Bart_preTrained")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [ ]:
#For saving it's tokenizer after saving
tokenizer.save_pretrained(r"/content/drive/MyDrive/ClickBait/Model_Bart_Tokenizers")

('/content/drive/MyDrive/ClickBait/Model_Bart_Tokenizers/tokenizer_config.json',
 '/content/drive/MyDrive/ClickBait/Model_Bart_Tokenizers/special_tokens_map.json',
 '/content/drive/MyDrive/ClickBait/Model_Bart_Tokenizers/vocab.json',
 '/content/drive/MyDrive/ClickBait/Model_Bart_Tokenizers/merges.txt',
 '/content/drive/MyDrive/ClickBait/Model_Bart_Tokenizers/added_tokens.json',
 '/content/drive/MyDrive/ClickBait/Model_Bart_Tokenizers/tokenizer.json')